In [1]:
import os
import pandas as pd
import cv2
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from tensorflow.keras.applications import ResNet50
import tensorflow as tf
from keras.layers import Dense , Flatten  , Conv2D , MaxPooling2D , Dropout,BatchNormalization, ReLU, Add, GlobalAveragePooling2D,Input
from PIL import Image
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import tf2onnx
import onnx

c:\Users\youss\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
# Define the path to the main directory containing subdirectories of labeled images
main_directory = 'Data'

# Initialize lists to store image data and labels
image_data_list = []
labels = []

# Function to load and preprocess an image using OpenCV
def load_image(image_path, target_size=(64, 64)):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, target_size)
    return image

# Walk through the directory structure
for root, dirs, files in os.walk(main_directory):
    
    for file in files:
        if file.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            image_path = os.path.join(root, file)
            label = os.path.basename(root)
            image_data = load_image(image_path)

            image_data_list.append(image_data)
            labels.append(label)


image_data_list= np.array(image_data_list)
labels=np.array(labels)


In [3]:
images = image_data_list / 255.0
label_to_index = {label: idx for idx, label in enumerate(np.unique(labels))}
indexed_labels = np.array([label_to_index[label] for label in labels])

# One-hot encode the labels
num_classes = len(np.unique(indexed_labels))
y = to_categorical(indexed_labels, num_classes)

X_train, X_test, y_train, y_test = train_test_split(images, y, test_size=0.2, random_state=42)


In [4]:

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(64,64,3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(14, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))
model.summary()


Epoch 1/20
49/49 [==============================] - 13s 233ms/step - loss: 2.0986 - accuracy: 0.3372 - val_loss: 1.1800 - val_accuracy: 0.5556
Epoch 2/20
49/49 [==============================] - 11s 223ms/step - loss: 1.1755 - accuracy: 0.5767 - val_loss: 0.7677 - val_accuracy: 0.6873
Epoch 3/20
49/49 [==============================] - 10s 207ms/step - loss: 0.8153 - accuracy: 0.6926 - val_loss: 0.5614 - val_accuracy: 0.7881
Epoch 4/20
49/49 [==============================] - 10s 208ms/step - loss: 0.6826 - accuracy: 0.7430 - val_loss: 0.4759 - val_accuracy: 0.8398
Epoch 5/20
49/49 [==============================] - 11s 216ms/step - loss: 0.5983 - accuracy: 0.7683 - val_loss: 0.4426 - val_accuracy: 0.8475
Epoch 6/20
49/49 [==============================] - 11s 216ms/step - loss: 0.5050 - accuracy: 0.8071 - val_loss: 0.3836 - val_accuracy: 0.8372
Epoch 7/20
49/49 [==============================] - 11s 218ms/step - loss: 0.4508 - accuracy: 0.8233 - val_loss: 0.3848 - val_accuracy: 0.8088

In [5]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)

# Print test results
print(f'Test loss: {test_loss}')
print(f'Test accuracy: {test_accuracy}')

13/13 [==============================] - 0s 29ms/step - loss: 0.1966 - accuracy: 0.9276
Test loss: 0.1965695023536682
Test accuracy: 0.9276486039161682


In [6]:

def preprocess_image(img_path, target_size=(64, 64)):
    # Load the image
    img = image.load_img(img_path, target_size=target_size)
    # Convert the image to a numpy array
    img_array = image.img_to_array(img)
    # Normalize the image
    img_array = img_array / 255.0
    # Add batch dimension
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

# Path to the image
img_path = 'D:/Hackathon/Dell-AI-Hackathon-2024/Data/Tomato(1-5)/frame190.jpg'

# Preprocess the image
preprocessed_image = preprocess_image(img_path)

In [7]:
predictions = model.predict(preprocessed_image)

# Get the class index with the highest probability
predicted_class_index = np.argmax(predictions[0])

# Output the prediction
print(f'Predicted class index: {predicted_class_index}')
print(label_to_index)

1/1 [==============================] - 0s 103ms/step
Predicted class index: 10
{'Apple(1-5)': 0, 'Apple(10-14)': 1, 'Apple(5-10)': 2, 'Banana(1-5)': 3, 'Banana(10-15)': 4, 'Banana(15-20)': 5, 'Banana(5-10)': 6, 'Carrot(1-2)': 7, 'Carrot(3-4)': 8, 'Expired': 9, 'Tomato(1-5)': 10, 'Tomato(10-15)': 11, 'Tomato(5-10)': 12, 'carrot(5-6)': 13}


In [8]:
onnx_model, _ = tf2onnx.convert.from_keras(model, opset=13)
onnx.save(onnx_model,  "Shelf_life.onnx")